In [ ]:
%load_ext autoreload
%autoreload 2
# %matplotlib widget
%pdb off

from pyCascade.cti import lesCreateImage as lcm
from pyCascade.cti import lesCreateMovie as vcm
import os
from matplotlib import pyplot as plt
import cv2 as cv
import numpy as np
import scipy as sp

############ Universal ################
home_dir = !pwd
display(home_dir)

In [ ]:
def process_flow_image(image_path, max=100):
    image = vcm.process_image(image_path, varlist=['planar'], cmaplist=None, data_min=[0], data_max=[max])
    image *= max
    background_value = sp.stats.mode(image, axis=None).mode
    non_bg = (image != background_value)
    if non_bg.size > 0:
        # Find the indices of non-background pixels
        rows = np.where(non_bg.any(axis=1))[0]
        cols = np.where(non_bg.any(axis=0))[0]

        if rows.size and cols.size:
            y_min, y_max = rows[0], rows[-1]
            x_min, x_max = cols[0], cols[-1]
            image = image[y_min:y_max+1, x_min:x_max+1]
        print("Cropped image shape:", image.shape)
    fig, ax = plt.subplots(1, 2, figsize=(8, 4))
    ax[1].imshow(image)

    horizontal_avg = np.mean(image, axis=1)
    ax[0].plot(horizontal_avg, np.arange(horizontal_avg.size, 0, -1))
    label = image_path.split('/')[-1].split('.')[0]
    ax[0].set_ylabel("Row index")
    ax[0].set_xlabel(label)
    ax[0].set_title("Horizontal Average Profile")
    return

qois = ["avg_u", "mag_u", "rms_p", "rms_u", "rms_v"]
case = "LRB_Cat_B_geometric_1to100"
for qoi in qois:
    image_path = f"{case}/image/0p6_{qoi}.00100000.png"
    process_flow_image(image_path, max=100)